Banking Fraud Detection System with Generative AI

## Complete End-to-End Solution

This notebook contains:
- **Embedding-Based Anomaly Detection** (4 ML models)
- **Natural Language Explanations** (GPT-4 powered or Generate Manually)
- **Banking-Specific Features** (37 features, 13 categories, 19 fraud patterns)
- **BSA/AML Compliance** (Velocity checks)

---

Step 1: Install Required Packages

In [ ]:
#!pip install -r requirements.txt

Step 2: Configuration & Imports

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
from collections import defaultdict
import warnings
import ssl
warnings.filterwarnings('ignore')

# ML imports
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# API imports
from openai import AzureOpenAI
# Load environment variables (force override)
from dotenv import load_dotenv
load_dotenv(override=True)

# Configuration
ANOMALY_THRESHOLD = float(os.getenv('ANOMALY_THRESHOLD', '0.70'))  # Banking-optimized
USE_AZURE = os.getenv('USE_AZURE', 'false').lower() == 'true'
USE_LLaMA = os.getenv('USE_LLaMA', 'false').lower() == 'true'
LLAMA_API_KEY = os.getenv('LLAMA_API_KEY', '')
LLAMA_API_BASE_URL = os.getenv('LLAMA_API_BASE_URL', 'https://api.llama.com/v1/')

print(f"Anomaly Threshold: {ANOMALY_THRESHOLD}")
print(f"Using Azure OpenAI: {USE_AZURE}")
print(f"Using LLaMA API: {USE_LLaMA}")

print("All packages imported successfully!")

Step 3: Loading banking transactions dataset

**Dataset Split Strategy (80/20):**
- **Training**: 80% of legitimate transactions (~3,300) - teaches model what "normal" looks like
- **Testing**: 20% of legitimate (~826) + ALL fraudulent (874) = ~1,700 total
- **Coverage**: Uses 100% of the 5,000 transaction dataset (no waste!)

In [ ]:
# Load dataset from JSON file
print("Loading banking transactions dataset...")

with open('data/banking_transactions.json', 'r') as f:
    all_transactions = json.load(f)

# Split into training (legitimate only) and test sets
# Use 80/20 split on ALL data (uses 100% of dataset)
legitimate_txns = [t for t in all_transactions if not t['is_fraud']]
fraudulent_txns = [t for t in all_transactions if t['is_fraud']]

# 80% of legitimate transactions for training
train_size_legit = int(0.8 * len(legitimate_txns))

# Training: 80% of legitimate transactions only (anomaly detection learns from normal data)
training_data = legitimate_txns[:train_size_legit]

# Testing: remaining 20% of legitimate + ALL fraudulent transactions
test_transactions = legitimate_txns[train_size_legit:] + fraudulent_txns

print(f"Loaded dataset:")
print(f"   - Total transactions: {len(all_transactions):,}")
print(f"   - Legitimate: {len(legitimate_txns):,} | Fraudulent: {len(fraudulent_txns):,}")
print(f"\nData Split (80/20):")
print(f"   - Training set: {len(training_data):,} transactions (legitimate only)")
print(f"   - Test set: {len(test_transactions):,} transactions")
print(f"     - Legitimate: {len(legitimate_txns[train_size_legit:]):,}")
print(f"     - Fraudulent: {len(fraudulent_txns):,}")
print(f"   - Test fraud rate: {len(fraudulent_txns)/len(test_transactions):.1%}")
print(f"\nUsing 100% of dataset (no data wasted!)")

Step 4: Banking Data Processor
Handles:
- 13 merchant categories (ATM, Wire, Gambling, etc.)
- 19 high-risk patterns
- Transaction type classification
- Velocity checks (BSA/AML compliance)

In [ ]:
class BankingDataProcessor:
    """Process banking transactions with industry-specific features"""
    
    def __init__(self):
        # 13 Banking merchant categories
        self.banking_categories = {
            'atm': ['atm', 'cash withdrawal', 'cashpoint'],
            'wire_transfer': ['wire', 'international transfer', 'remittance', 'swift'],
            'bill_payment': ['utility', 'bill pay', 'payment service'],
            'gambling': ['casino', 'gambling', 'betting', 'lottery', 'poker'],
            'cash_advance': ['cash advance', 'payday', 'quick cash'],
            'investment': ['brokerage', 'investment', 'trading', 'securities'],
            'p2p': ['venmo', 'paypal', 'zelle', 'cash app', 'peer to peer'],
            'check': ['check cashing', 'check deposit', 'mobile deposit'],
            'loan': ['loan payment', 'mortgage', 'credit line'],
            'insurance': ['insurance', 'premium payment'],
            'government': ['irs', 'tax payment', 'government fee'],
            'crypto': ['crypto', 'bitcoin', 'coinbase', 'blockchain'],
            'retail': ['store', 'shop', 'market', 'purchase']
        }
        
        # 19 High-risk patterns
        self.high_risk_patterns = [
            'wire', 'transfer', 'international', 'offshore', 'crypto', 'bitcoin',
            'gambling', 'casino', 'betting', 'cash advance', 'payday',
            'gift card', 'prepaid', 'money order', 'western union',
            'check cashing', 'pawn', 'atm', 'withdrawal'
        ]
        
        # Transaction types
        self.transaction_types = {
            'debit': ['purchase', 'payment', 'withdrawal', 'debit'],
            'credit': ['refund', 'credit', 'deposit', 'return'],
            'transfer': ['transfer', 'wire', 'ach'],
            'withdrawal': ['withdrawal', 'atm', 'cash'],
            'deposit': ['deposit', 'mobile deposit'],
            'payment': ['bill', 'payment', 'utility']
        }
        
        # Velocity thresholds (BSA/AML)
        self.velocity_thresholds = {
            'hourly_count': 10,
            'daily_count': 50,
            'hourly_amount': 10000,
            'daily_amount': 50000
        }
        
        self.user_histories = defaultdict(list)
    
    def extract_transaction_type(self, description: str) -> str:
        """Classify transaction type"""
        desc_lower = description.lower()
        for txn_type, keywords in self.transaction_types.items():
            if any(keyword in desc_lower for keyword in keywords):
                return txn_type
        return 'other'
    
    def check_velocity(self, user_id: str, timestamp: datetime, amount: float) -> Dict[str, Any]:
        """Check velocity limits for BSA/AML compliance"""
        user_txns = self.user_histories[user_id]
        
        # Count transactions in last hour and day
        one_hour_ago = timestamp - timedelta(hours=1)
        one_day_ago = timestamp - timedelta(days=1)
        
        hourly_txns = [t for t in user_txns if t['timestamp'] >= one_hour_ago]
        daily_txns = [t for t in user_txns if t['timestamp'] >= one_day_ago]
        
        hourly_count = len(hourly_txns)
        daily_count = len(daily_txns)
        hourly_amount = sum(t['amount'] for t in hourly_txns)
        daily_amount = sum(t['amount'] for t in daily_txns)
        
        return {
            'hourly_count': hourly_count,
            'daily_count': daily_count,
            'hourly_amount': hourly_amount,
            'daily_amount': daily_amount,
            'count_breach_hourly': hourly_count > self.velocity_thresholds['hourly_count'],
            'count_breach_daily': daily_count > self.velocity_thresholds['daily_count'],
            'amount_breach_hourly': hourly_amount > self.velocity_thresholds['hourly_amount'],
            'amount_breach_daily': daily_amount > self.velocity_thresholds['daily_amount']
        }
    
    def extract_banking_features(self, transaction: Dict[str, Any]) -> Dict[str, bool]:
        """Extract 12 banking-specific boolean features"""
        desc_lower = transaction['description'].lower()
        merchant_lower = transaction.get('merchant', '').lower()
        
        return {
            'is_atm': any(k in desc_lower or k in merchant_lower for k in self.banking_categories['atm']),
            'is_wire': any(k in desc_lower or k in merchant_lower for k in self.banking_categories['wire_transfer']),
            'is_gambling': any(k in desc_lower or k in merchant_lower for k in self.banking_categories['gambling']),
            'is_cash_advance': any(k in desc_lower or k in merchant_lower for k in self.banking_categories['cash_advance']),
            'is_p2p': any(k in desc_lower or k in merchant_lower for k in self.banking_categories['p2p']),
            'is_crypto': any(k in desc_lower or k in merchant_lower for k in self.banking_categories['crypto']),
            'is_check': any(k in desc_lower or k in merchant_lower for k in self.banking_categories['check']),
            'is_card_present': transaction.get('card_present', False),
            'is_card_not_present': not transaction.get('card_present', True),
            'is_cross_border': transaction.get('location', '').lower() in ['international', 'foreign', 'offshore'],
            'is_mobile': 'mobile' in desc_lower or 'app' in desc_lower,
            'has_high_risk_pattern': any(pattern in desc_lower or pattern in merchant_lower 
                                         for pattern in self.high_risk_patterns)
        }
    
    def process_transaction(self, transaction: Dict[str, Any]) -> Dict[str, Any]:
        """Process a single banking transaction"""
        user_id = transaction['user_id']
        timestamp = datetime.fromisoformat(transaction['timestamp'])
        amount = transaction['amount']
        
        # Extract features
        txn_type = self.extract_transaction_type(transaction['description'])
        velocity = self.check_velocity(user_id, timestamp, amount)
        banking_features = self.extract_banking_features(transaction)
        
        # Categorize merchant
        merchant_lower = transaction.get('merchant', '').lower()
        desc_lower = transaction['description'].lower()
        category = 'other'
        for cat, keywords in self.banking_categories.items():
            if any(k in merchant_lower or k in desc_lower for k in keywords):
                category = cat
                break
        
        # Build processed transaction
        processed = {
            **transaction,
            'transaction_type': txn_type,
            'merchant_category': category,
            'velocity': velocity,
            'banking_features': banking_features,
            'timestamp_dt': timestamp
        }
        
        # Update history
        self.user_histories[user_id].append({
            'timestamp': timestamp,
            'amount': amount
        })
        
        return processed

print("Banking Data Processor ready!")

Step 5: Anomaly Detection System
Uses 4 ML models:
1. Isolation Forest
2. Local Outlier Factor
3. One-Class SVM
4. Mahalanobis Distance

In [ ]:
class EmbeddingAnomalyDetector:
    """Multi-model anomaly detection with embeddings"""
    
    def __init__(self, model_name: str = 'models/all-MiniLM-L6-v2'):
        """
        Initialize detector with local embedding model
        Args:
            model_name: Path to local model folder (default: 'models/all-MiniLM-L6-v2')
        """
        print(f"Loading embedding model from: {model_name}")
        try:
            # Try to load from local folder first
            self.embedding_model = SentenceTransformer(model_name, device='cpu')
            print(f"Loaded model from local folder: {model_name}")
        except Exception as e:
            print(f"Failed to load from {model_name}: {e}")
            print("Attempting to download 'all-MiniLM-L6-v2' from HuggingFace...")
            try:
                # Fallback: download from HuggingFace (with SSL disabled)
                self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')
                print("Model downloaded and loaded successfully")
            except Exception as e2:
                print(f"Failed to download model: {e2}")
                raise RuntimeError("Could not load embedding model. Please check 'models' folder or internet connection.")
        
        self.scaler = StandardScaler()
        self.pca = PCA(n_components=0.95)
        
        # 4 ML models
        # The Mahalanobis Distance model is different from the other three. 
        # It's not a scikit-learn class hence we dont need to initialize. 
        self.isolation_forest = IsolationForest(contamination=0.1, random_state=42, n_estimators=100)
        self.lof = LocalOutlierFactor(contamination=0.1, novelty=True, n_neighbors=20)
        self.one_class_svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.1)
        
        self.is_trained = False
        self.user_histories = defaultdict(list)
    
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """Generate embeddings from text descriptions"""
        return self.embedding_model.encode(texts, show_progress_bar=False)
    
    def extract_numerical_features(self, transaction: Dict[str, Any]) -> np.ndarray:
        """Extract 37 numerical features for banking"""
        user_id = transaction['user_id']
        amount = transaction['amount']
        timestamp = transaction['timestamp_dt']
        
        # User history stats
        user_amounts = self.user_histories.get(user_id, [amount])
        mean_amount = np.mean(user_amounts)
        median_amount = np.median(user_amounts)
        max_amount = np.max(user_amounts)
        p95_amount = np.percentile(user_amounts, 95)
        
        # 37 Features
        features = [
            # Amount features (9)
            amount,
            np.log1p(amount),
            1 if amount % 100 == 0 else 0,  # Round number
            1 if amount > 1000 else 0,
            1 if amount > 5000 else 0,
            amount / (mean_amount + 1),
            amount / (median_amount + 1),
            amount / (max_amount + 1),
            1 if amount > p95_amount else 0,
            
            # Temporal features (7)
            timestamp.hour,
            timestamp.weekday(),
            timestamp.day,
            1 if timestamp.weekday() >= 5 else 0,  # Weekend
            1 if timestamp.hour < 6 or timestamp.hour >= 22 else 0,  # Night
            1 if 9 <= timestamp.hour <= 17 else 0,  # Business hours
            timestamp.timestamp(),
            
            # Location features (3)
            1 if transaction.get('location', '').lower() in ['online', 'internet'] else 0,
            1 if transaction.get('location', '').lower() == 'international' else 0,
            1 if transaction['banking_features']['is_cross_border'] else 0,
            
            # Banking-specific features (10)
            1 if transaction['banking_features']['is_atm'] else 0,
            1 if transaction['banking_features']['is_gambling'] else 0,
            1 if transaction['banking_features']['is_cash_advance'] else 0,
            1 if transaction['banking_features']['is_wire'] else 0,
            1 if transaction['banking_features']['is_p2p'] else 0,
            1 if transaction['banking_features']['is_card_present'] else 0,
            1 if transaction['banking_features']['is_card_not_present'] else 0,
            1 if transaction['banking_features']['is_check'] else 0,
            1 if transaction['banking_features']['is_mobile'] else 0,
            1 if transaction['banking_features']['has_high_risk_pattern'] else 0,
            
            # Velocity features (8)
            transaction['velocity']['hourly_count'],
            transaction['velocity']['daily_count'],
            transaction['velocity']['hourly_amount'],
            transaction['velocity']['daily_amount'],
            1 if transaction['velocity']['count_breach_hourly'] else 0,
            1 if transaction['velocity']['count_breach_daily'] else 0,
            1 if transaction['velocity']['amount_breach_hourly'] else 0,
            1 if transaction['velocity']['amount_breach_daily'] else 0,
        ]
        
        return np.array(features, dtype=float)
    
    def train(self, transactions: List[Dict[str, Any]]):
        """Train all 4 models"""
        print(f"Training on {len(transactions)} transactions...")
        
        # Build user histories
        for txn in transactions:
            user_id = txn['user_id']
            self.user_histories[user_id].append(txn['amount'])
        
        # Generate embeddings
        texts = [f"{t['description']} {t.get('merchant', '')}" for t in transactions]
        embeddings = self.generate_embeddings(texts)
        
        # Extract numerical features
        numerical_features = np.array([self.extract_numerical_features(t) for t in transactions])
        
        # Combine features
        combined_features = np.hstack([embeddings, numerical_features])
        
        # Scale and reduce dimensions
        scaled_features = self.scaler.fit_transform(combined_features)
        reduced_features = self.pca.fit_transform(scaled_features)
        
        # Train models
        self.isolation_forest.fit(reduced_features)
        self.lof.fit(reduced_features)
        self.one_class_svm.fit(reduced_features)
        
        # Store for Mahalanobis
        #self.mean = The center point (mean) of all legitimate transactions
        #self.cov_inv = The inverse covariance matrix (how features vary together)
        self.mean = np.mean(reduced_features, axis=0)
        self.cov_inv = np.linalg.pinv(np.cov(reduced_features.T))
        
        self.is_trained = True
        print("All models trained!")
    
    def predict_single(self, transaction: Dict[str, Any]) -> Tuple[float, Dict[str, float]]:
        """Predict anomaly score for single transaction"""
        if not self.is_trained:
            raise ValueError("Models not trained yet!")
        
        # Generate features
        text = f"{transaction['description']} {transaction.get('merchant', '')}"
        embedding = self.generate_embeddings([text])[0]
        numerical = self.extract_numerical_features(transaction)
        
        # Combine and transform
        combined = np.hstack([embedding, numerical]).reshape(1, -1)
        scaled = self.scaler.transform(combined)
        reduced = self.pca.transform(scaled)
        
        # Get scores from all 4 models
        if_score = -self.isolation_forest.score_samples(reduced)[0]
        lof_score = -self.lof.score_samples(reduced)[0]
        svm_score = -self.one_class_svm.score_samples(reduced)[0]
        
        # Mahalanobis distance
        diff = reduced[0] - self.mean
        mahal_dist = np.sqrt(diff @ self.cov_inv @ diff.T)
        
        # Normalize scores to 0-1
        scores = {
            'isolation_forest': min(max(if_score / 2 + 0.5, 0), 1),
            'local_outlier_factor': min(max(lof_score / 2 + 0.5, 0), 1),
            'one_class_svm': min(max(svm_score / 2 + 0.5, 0), 1),
            'mahalanobis': min(max(mahal_dist / 10, 0), 1)
        }
        
        # Ensemble score (average)
        ensemble_score = np.mean(list(scores.values()))
        
        return ensemble_score, scores

print("Anomaly Detector ready!")

Step 6: Natural Language Explanation Generator

Uses GPT-4 to generate human-readable fraud explanations or Generate explanation

In [ ]:
class ExplanationGenerator:
    """Generate natural language explanations for fraud alerts"""
    
    def __init__(self, use_azure: bool = True, use_llama: bool = False):
        """Initialize explanation generator
        
        Args:
            use_azure: If True, use custom Azure OpenAI with OAuth2 authentication
            use_llama: If True, use LLaMA API for explanations
            If both False, use template-based explanations
        """
        self.use_azure = use_azure
        self.use_llama = use_llama
        
        if use_azure:
            print("Using custom Azure OpenAI with OAuth2 authentication")
        elif use_llama:
            print("Using LLaMA API for explanations")
        else:
            print("Using template-based explanations")
    
    def generate_explanation(self, transaction: Dict[str, Any], anomaly_score: float, 
                           model_scores: Dict[str, float]) -> str:
        """Generate fraud explanation"""
        
        # Build context
        risk_factors = []
        if transaction['amount'] > 5000:
            risk_factors.append(f"Large amount (${transaction['amount']:,.2f})")
        if transaction['timestamp_dt'].hour < 6 or transaction['timestamp_dt'].hour >= 22:
            risk_factors.append("Night-time transaction")
        if transaction['banking_features']['is_wire']:
            risk_factors.append("Wire transfer")
        if transaction['banking_features']['is_gambling']:
            risk_factors.append("Gambling merchant")
        if transaction['banking_features']['is_cross_border']:
            risk_factors.append("International/Cross-border")
        if transaction['velocity']['count_breach_hourly']:
            risk_factors.append("Velocity breach (hourly)")
        
        # Try Azure OpenAI if enabled
        if self.use_azure:
            try:
                from azure_openai import azure_chat
                import asyncio
                
                prompt = f"""You are a banking fraud analyst. Explain why this transaction was flagged as suspicious.

Transaction Details:
- Description: {transaction['description']}
- Merchant: {transaction.get('merchant', 'N/A')}
- Amount: ${transaction['amount']:,.2f}
- Category: {transaction['merchant_category']}
- Type: {transaction['transaction_type']}
- Time: {transaction['timestamp_dt'].strftime('%Y-%m-%d %I:%M %p')}
- Location: {transaction.get('location', 'N/A')}

Risk Score: {anomaly_score:.2%}
Risk Factors: {', '.join(risk_factors) if risk_factors else 'None specific'}
Model Scores: {', '.join(f'{k}: {v:.2f}' for k, v in model_scores.items())}

Provide a clear, concise explanation (2-3 sentences) and recommend specific actions."""
                
                # Call custom Azure OpenAI function (async)
                try:
                    # Try to get the current event loop
                    loop = asyncio.get_event_loop()
                    if loop.is_running():
                        # We're in Jupyter - use nest_asyncio
                        import nest_asyncio
                        nest_asyncio.apply()
                        response_content = asyncio.run(azure_chat(prompt))
                    else:
                        response_content = asyncio.run(azure_chat(prompt))
                except:
                    # Fallback: create new event loop
                    response_content = asyncio.run(azure_chat(prompt))
                
                return response_content
                
            except Exception as e:
                print(f"Azure OpenAI call failed (falling back to template): {e}")
        
        # Try LLaMA API if enabled
        elif self.use_llama:
            try:
                from openai import OpenAI
                
                # Validate API credentials
                if not LLAMA_API_KEY or LLAMA_API_KEY.strip() == '':
                    raise ValueError("LLAMA_API_KEY is not configured. Please set it in .env file")
                
                # Initialize LLaMA API client
                client = OpenAI(
                    api_key=LLAMA_API_KEY,
                    base_url=LLAMA_API_BASE_URL,
                )
                
                prompt = f"""You are a banking fraud analyst. Explain why this transaction was flagged as suspicious.

Transaction Details:
- Description: {transaction['description']}
- Merchant: {transaction.get('merchant', 'N/A')}
- Amount: ${transaction['amount']:,.2f}
- Category: {transaction['merchant_category']}
- Type: {transaction['transaction_type']}
- Time: {transaction['timestamp_dt'].strftime('%Y-%m-%d %I:%M %p')}
- Location: {transaction.get('location', 'N/A')}

Risk Score: {anomaly_score:.2%}
Risk Factors: {', '.join(risk_factors) if risk_factors else 'None specific'}
Model Scores: {', '.join(f'{k}: {v:.2f}' for k, v in model_scores.items())}

Provide a clear, concise explanation (2-3 sentences) and recommend specific actions."""
                
                # Call LLaMA API with proper error handling
                response = client.chat.completions.create(
                    messages=[
                    {"role": "assistant", "content": prompt},
                    ],
                    model="Llama-3.3-8B-Instruct",
                    temperature=0.6,
                    max_completion_tokens=2048,
                    top_p=0.9,
                    frequency_penalty=1
                )
                # Check if response is valid and has the expected structure
                # Validate response
                if response is None:
                    raise ValueError("LLaMA API returned None response")

                # LLaMA returns completion_message instead of choices
                if hasattr(response, "completion_message") and response.completion_message:
                    content_block = response.completion_message.get("content")

                    if isinstance(content_block, dict) and content_block.get("type") == "text":
                        content = content_block.get("text")

                        if not content or content.strip() == "":
                            raise ValueError("LLaMA API returned empty content")

                        return content

                    else:
                        raise ValueError("Unexpected LLaMA content format")

                else:
                    raise ValueError("LLaMA API response missing completion_message")

                
            except Exception as e:
                print(f"LLaMA API call failed (falling back to template): {e}")
        
        # Fallback template-based explanation
        explanation = f"""**Fraud Alert**

This transaction scored {anomaly_score:.1%} on our fraud detection models.

**Risk Factors Detected:**
{chr(10).join(f'• {factor}' for factor in risk_factors) if risk_factors else '• Multiple anomalous patterns detected'}

**Model Analysis:**
• Isolation Forest: {model_scores['isolation_forest']:.2f}
• Local Outlier Factor: {model_scores['local_outlier_factor']:.2f}
• One-Class SVM: {model_scores['one_class_svm']:.2f}
• Mahalanobis Distance: {model_scores['mahalanobis']:.2f}

**Recommended Actions:**
• Contact account holder immediately for verification
• Review recent account activity for similar patterns
• Consider temporary hold pending confirmation
• Document for compliance review (SAR/CTR if applicable)
"""
        return explanation

print("Explanation Generator ready!")

Step 7: Complete Fraud Detection System

Orchestrates all components

In [ ]:
class BankingFraudDetectionSystem:
    """Complete banking fraud detection system"""
    
    def __init__(self, threshold: float = 0.70, use_azure: bool = True, use_llama: bool = False):
        self.processor = BankingDataProcessor()
        self.detector = EmbeddingAnomalyDetector()
        self.explainer = ExplanationGenerator(use_azure=use_azure, use_llama=use_llama)
        self.threshold = threshold
        self.results = []
    
    def train(self, transactions: List[Dict[str, Any]]):
        """Train the system"""
        print(f"\nTraining Banking Fraud Detection System...")
        processed_txns = [self.processor.process_transaction(t) for t in transactions]
        self.detector.train(processed_txns)
        print(f"System trained on {len(transactions)} transactions\n")
    
    def analyze_transaction(self, transaction: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze single transaction"""
        # Process
        processed = self.processor.process_transaction(transaction)
        
        # Detect
        anomaly_score, model_scores = self.detector.predict_single(processed)
        
        # Determine if fraud
        is_fraud = anomaly_score >= self.threshold
        
        # Generate explanation if fraud
        explanation = None
        if is_fraud:
            explanation = self.explainer.generate_explanation(processed, anomaly_score, model_scores)
        
        # Categorize risk
        if anomaly_score >= 0.9:
            risk_level = "CRITICAL"
        elif anomaly_score >= 0.75:
            risk_level = "HIGH"
        elif anomaly_score >= 0.5:
            risk_level = "MEDIUM"
        else:
            risk_level = "LOW"
        
        result = {
            'transaction': transaction,
            'processed': processed,
            'is_fraud': is_fraud,
            'anomaly_score': anomaly_score,
            'model_scores': model_scores,
            'risk_level': risk_level,
            'explanation': explanation
        }
        
        self.results.append(result)
        return result
    
    def get_summary(self) -> Dict[str, Any]:
        """Get fraud detection summary"""
        if not self.results:
            return {}
        
        total = len(self.results)
        fraud_count = sum(1 for r in self.results if r['is_fraud'])
        
        return {
            'total_transactions': total,
            'fraud_detected': fraud_count,
            'fraud_rate': fraud_count / total if total > 0 else 0,
            'avg_score': np.mean([r['anomaly_score'] for r in self.results]),
            'risk_distribution': {
                'CRITICAL': sum(1 for r in self.results if r['risk_level'] == 'CRITICAL'),
                'HIGH': sum(1 for r in self.results if r['risk_level'] == 'HIGH'),
                'MEDIUM': sum(1 for r in self.results if r['risk_level'] == 'MEDIUM'),
                'LOW': sum(1 for r in self.results if r['risk_level'] == 'LOW')
            }
        }

print("Complete Fraud Detection System ready!")

Step 8: Train the System

Train the fraud detection system on 80% of legitimate transactions (~3,300) from the dataset

In [ ]:
# Initialize system
fraud_system = BankingFraudDetectionSystem(
    threshold=ANOMALY_THRESHOLD,
    use_azure=USE_AZURE,
    use_llama=USE_LLaMA
)

# Train on normal transactions
fraud_system.train(training_data)

Step 9: Analyze Test Transactions

Run fraud detection on test transactions and compare with ground truth

In [ ]:
"""
# Analyze test transactions
print("\n" + "="*80)
print("ANALYZING TEST TRANSACTIONS")
print("="*80 + "\n")

# Process all transactions
print(f"Processing {len(test_transactions)} transactions...\n")
for txn in test_transactions:
    fraud_system.analyze_transaction(txn)

# Show only FRAUD cases (reduces output by ~50%)
print("\n" + "="*80)
print("FRAUD ALERTS DETECTED")
print("="*80)

fraud_results = [r for r in fraud_system.results if r['is_fraud']]

if fraud_results:
    print(f"\nFound {len(fraud_results)} fraudulent transactions:\n")
    
    for result in fraud_results:
        txn = result['transaction']
        print(f"\n{'='*80}")
        print(f"Transaction: {txn['transaction_id']}")
        print(f"{'='*80}")
        print(f"Description: {txn['description']}")
        print(f"Merchant: {txn['merchant']}")
        print(f"Amount: ${txn['amount']:,.2f}")
        print(f"Time: {datetime.fromisoformat(txn['timestamp']).strftime('%Y-%m-%d %I:%M %p')}")
        print(f"Location: {txn['location']}")
        print(f"\nFRAUD DETECTION RESULT:")
        print(f"   Risk Score: {result['anomaly_score']:.1%}")
        print(f"   Risk Level: {result['risk_level']}")
        print(f"   Fraud Status: FRAUD DETECTED")
        print(f"\nModel Scores:")
        for model, score in result['model_scores'].items():
            print(f"   • {model.replace('_', ' ').title()}: {score:.2f}")
        
        if result['explanation']:
            print(f"\nEXPLANATION:\n")
            print(result['explanation'])
        
        print("\n")
else:
    print("\nNo fraud detected in test set!")

print(f"\nTip: See full analysis (including legitimate transactions) in the summary below.")

"""

Step 10: Model Performance & Summary

Evaluate model performance with ground truth labels and view statistics

In [ ]:
"""
# Get summary
summary = fraud_system.get_summary()

print("\n" + "="*80)
print("FRAUD DETECTION SUMMARY")
print("="*80)
print(f"\nTotal Transactions Analyzed: {summary['total_transactions']}")
print(f"Fraud Detected: {summary['fraud_detected']}")
print(f"Fraud Rate: {summary['fraud_rate']:.1%}")
print(f"Average Risk Score: {summary['avg_score']:.1%}")

print(f"\nRisk Level Distribution:")
for level, count in summary['risk_distribution'].items():
    print(f"   {level}: {count} transactions")

# Model Performance Evaluation (compare with ground truth)
print("\n" + "="*80)
print("MODEL PERFORMANCE EVALUATION")
print("="*80)

y_true = [r['transaction']['is_fraud'] for r in fraud_system.results]
y_pred = [r['is_fraud'] for r in fraud_system.results]

# Calculate metrics
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

print(f"\nConfusion Matrix:")
print(f"   True Negatives (Correct Legit):  {tn:4d}")
print(f"   False Positives (False Alarms):  {fp:4d}")
print(f"   False Negatives (Missed Fraud):  {fn:4d}")
print(f"   True Positives (Caught Fraud):   {tp:4d}")

print(f"\nPerformance Metrics:")
print(f"   Accuracy:  {accuracy:.2%}")
print(f"   Precision: {precision:.2%} (of flagged, how many are actually fraud)")
print(f"   Recall:    {recall:.2%} (of actual fraud, how many we caught)")
print(f"   F1 Score:  {f1:.2%} (balanced precision-recall)")

print("\n" + "="*80)

# Create DataFrame for better visualization
results_df = pd.DataFrame([{
    'Transaction ID': r['transaction']['transaction_id'],
    'Merchant': r['transaction']['merchant'][:25],
    'Amount': f"${r['transaction']['amount']:,.2f}",
    'Risk Score': f"{r['anomaly_score']:.1%}",
    'Predicted': 'FRAUD' if r['is_fraud'] else 'OK',
    'Actual': 'FRAUD' if r['transaction']['is_fraud'] else 'OK',
    'Match': 'YES' if r['is_fraud'] == r['transaction']['is_fraud'] else 'NO'
} for r in fraud_system.results[:100]])  # Show first 20

print("\nSAMPLE RESULTS (First 100):\n")
print(results_df.to_string(index=False))
print("\n... and more\n")

"""

Manual Testing - Test Your Own Transactions

Use this cell to manually test any transaction and see fraud detection in action!

In [ ]:
def test_transaction(description, merchant, amount, location="Online", hour=14):
    """
    Quick function to test a transaction manually
    
    Parameters:
    -----------
    description : str
        Transaction description (e.g., "Bitcoin purchase")
    merchant : str
        Merchant name (e.g., "Crypto.com")
    amount : float
        Transaction amount (e.g., 8500.00)
    location : str, optional
        Location (e.g., "Online", "International", "New York")
    hour : int, optional
        Hour of day 0-23 (default: 14 = 2 PM)
    """
    
    # Create timestamp
    now = datetime.now()
    test_time = now.replace(hour=hour, minute=0, second=0)
    
    # Build transaction
    test_txn = {
        'transaction_id': 'TEST001',
        'user_id': 'test_user',
        'amount': amount,
        'merchant': merchant,
        'description': description,
        'timestamp': test_time.isoformat(),
        'location': location,
        'card_present': False,
        'is_fraud': False  # We're testing, so this is unknown
    }
    
    # Analyze
    print("="*80)
    print("TESTING YOUR TRANSACTION")
    print("="*80)
    print(f"\nTransaction Details:")
    print(f"   Description: {description}")
    print(f"   Merchant: {merchant}")
    print(f"   Amount: ${amount:,.2f}")
    print(f"   Location: {location}")
    print(f"   Time: {test_time.strftime('%I:%M %p')}")
    
    # Run fraud detection
    result = fraud_system.analyze_transaction(test_txn)
    
    # Display results
    print(f"\n{'='*80}")
    print(f"FRAUD DETECTION RESULT")
    print(f"{'='*80}")
    print(f"\n   Risk Score: {result['anomaly_score']:.1%}")
    print(f"   Risk Level: {result['risk_level']}")
    
    if result['is_fraud']:
        print(f"   Status: FRAUD DETECTED")
    else:
        print(f"   Status: LEGITIMATE")
    
    print(f"\nModel Scores:")
    for model_name, score in result['model_scores'].items():
        print(f"   - {model_name.replace('_', ' ').title()}: {score:.2f}")
    
    if result['explanation']:
        print(f"\nEXPLANATION:")
        print("="*80)
        print(result['explanation'])
    
    print("\n" + "="*80)
    
    return result

print("Manual testing function loaded!")
print("\nUsage examples:")
print("  test_transaction('Bitcoin purchase', 'Crypto.com', 8500, 'International', 2)")
print("  test_transaction('Grocery shopping', 'Walmart', 127.50, 'New York', 14)")
print("  test_transaction('Wire transfer', 'Bank', 15000, 'International', 23)")

Create Your Own Test - Edit and Run!

**Instructions:**
1. Change the values below to test your own transaction
2. Run the cell to see fraud detection result
3. Try different combinations to understand the model!

🎯 Test Fraud Detection with Azure Explanations or Manual Template explanation

Now test a fraud transaction to see Azure OpenAI-generated explanation (make sure USE_AZURE=true in .env) or Template Based

In [ ]:
# Test fraud detection with LLM-generated explanation
print("="*80)
print("TESTING FRAUD DETECTION WITH LLM EXPLANATIONS")
print("="*80)

# High-risk transaction that should trigger fraud detection
result = test_transaction(
    description="Wire transfer to offshore account Bitcoin purchase",
    merchant="International Wire Service", 
    amount=12500.00,
    location="International",
    hour=2  # 2 AM
)

# Check which backend was used for explanation
print("\nLLM Backend Status:")
print(f"   Configuration:")
print(f"   - USE_AZURE: {USE_AZURE}")
print(f"   - USE_LLaMA: {USE_LLaMA}")

if result['explanation']:
    # Determine which backend was likely used based on explanation format
    is_template = "**Fraud Alert**" in result['explanation']
    
    if USE_AZURE:
        if is_template:
            print("\n   Status: Azure OpenAI was configured but call failed")
            print("   Fallback: Using template-based explanation")
        else:
            print("\n   Status: Successfully used Azure OpenAI (GPT-4)")
            print("   ✓ Natural language explanation generated")
    elif USE_LLaMA:
        if is_template:
            print("\n   Status: LLaMA API was configured but call failed")
            print("   Fallback: Using template-based explanation")
        else:
            print("\n   Status: Successfully used LLaMA API")
            print("   ✓ Natural language explanation generated")
    else:
        print("\n   Status: No LLM backend enabled")
        print("   Using: Template-based explanation")
        print("   To enable LLMs:")
        print("      - Set USE_AZURE=true for Azure OpenAI")
        print("      - Set USE_LLaMA=true for LLaMA API")
else:
    print("\n   Status: No explanation generated")

In [ ]:
# YOUR CUSTOM TEST - Edit these values and run!
test_transaction(
    description="Buy Bitcoin",  # e.g., "Coffee purchase", "Bitcoin trade"
    merchant="Coinbase",        # e.g., "Starbucks", "Coinbase"
    amount=100.00,                        # e.g., 50.00, 10000.00
    location="Online",                    # e.g., "New York", "International", "Online"
    hour=2                               # 0-23 (e.g., 2=2am, 14=2pm, 23=11pm)
)